# Notebook 5.8 - Final model and metrics, plus regression

# Import libraries

In [1]:
import xgboost as xgb
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from scipy import stats
from scipy.special import inv_boxcox
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
import statsmodels.api as sm

# Choose the city

In [2]:
# Choose the city ('Madrid', 'Barcelona', 'Valencia', or 'Combined')
city = 'Combined'

# Load the cleaned data set incl. the feature "type of road"

In [3]:
if city.lower() == 'madrid':
    data = pd.read_csv('../../data/5_cleaned_and_feature_engineering/feature_road_type/madrid_cleaned_incl_road_type.csv')
elif city.lower() == 'barcelona':
    data = pd.read_csv('../../data/5_cleaned_and_feature_engineering/feature_road_type/barcelona_cleaned_incl_road_type.csv')
elif city.lower() == 'valencia':
    data = pd.read_csv('../../data/5_cleaned_and_feature_engineering/feature_road_type/valencia_cleaned_incl_road_type.csv')
elif city.lower() == 'combined':
    #Read all 3 datasets and add a column to indicate source dataset
    madrid_df = pd.read_csv('../../data/5_cleaned_and_feature_engineering/feature_road_type/madrid_cleaned_incl_road_type.csv')
    valencia_df = pd.read_csv('../../data/5_cleaned_and_feature_engineering/feature_road_type/valencia_cleaned_incl_road_type.csv')
    barcelona_df = pd.read_csv('../../data/5_cleaned_and_feature_engineering/feature_road_type/barcelona_cleaned_incl_road_type.csv')
    madrid_df['Madrid'] = True
    madrid_df['Valencia'] = False
    madrid_df['Barcelona'] = False    
    valencia_df['Madrid'] = False
    valencia_df['Valencia'] = True
    valencia_df['Barcelona'] = False    
    barcelona_df['Madrid'] = False
    barcelona_df['Valencia'] = False
    barcelona_df['Barcelona'] = True
    data = pd.concat([madrid_df, valencia_df, barcelona_df], ignore_index=True)

In [4]:
data.head()

,ASSETID,PRICE,CONSTRUCTEDAREA,ROOMNUMBER,BATHNUMBER,AMENITYID,HASPARKINGSPACE,PARKINGSPACEPRICE,HASTERRACE,HASLIFT,...,PERIOD_201806,PERIOD_201809,PERIOD_201812,MOTORWAY/PRIMARY,OTHER,PEDESTRIAN,SECONDARY/TERTIARY,Madrid,Valencia,Barcelona
0,A10000037964896093228,255000,97,3,2,3,0,1.0,0,1,...,0,1,0,False,True,False,False,True,False,False
1,A10000072440601830803,82000,62,2,1,3,0,1.0,0,1,...,1,0,0,False,True,False,False,True,False,False
2,A10000538600815177437,133000,67,3,1,3,0,1.0,1,0,...,0,0,0,False,True,False,False,True,False,False
3,A10000654405436195291,204000,180,3,2,3,0,1.0,0,1,...,0,0,0,False,True,False,False,True,False,False
4,A10000872160480475600,161000,54,2,1,3,0,1.0,0,0,...,0,0,1,False,True,False,False,True,False,False


# Create clusters based on distance to city center as differentiation of metrics

In [5]:
# Function to categorize distance into clusters
def categorize_distance(df, city_column=None):
    if city_column:
        df_city = df[df[city_column]]
    else:
        df_city = df
    df_city = df_city.copy()  # Avoid SettingWithCopyWarning
    df_city['DISTANCE_CLUSTER'] = pd.qcut(df_city['DISTANCE_TO_CITY_CENTER'], 
                                          q=3, 
                                          labels=['LOW_DISTANCE_TO_CENTER', 'MEDIUM_DISTANCE_TO_CENTER', 'HIGH_DISTANCE_TO_CENTER'])
    return df_city[['DISTANCE_CLUSTER']]

# Check if the city is 'Combined'
if city.lower() == 'combined':
    # Filter the dataframe for each city and categorize distance
    data['DISTANCE_CLUSTER'] = None
    
    for city_name in ['Madrid', 'Barcelona', 'Valencia']:
        data_city_cluster = categorize_distance(data, city_name)
        data.loc[data[city_name] == True, 'DISTANCE_CLUSTER'] = data_city_cluster['DISTANCE_CLUSTER']
else:
    # Create distance clusters based on the "DISTANCE_TO_CITY_CENTER" column without differentiating between cities
    data_clustered = categorize_distance(data)
    data['DISTANCE_CLUSTER'] = data_clustered['DISTANCE_CLUSTER']

# Defining X and y

__Transform target variable PRICE as optimized in notebook 5.1__

In [6]:
# Define the lambda value
specified_lambda = 0

# Apply Box-Cox transformation using the specified lambda value
price_transformed = stats.boxcox(data['PRICE'], lmbda=specified_lambda)

# Add the transformed PRICE back to the DataFrame
data['PRICE_TRANSFORMED'] = price_transformed

__Assign features as X and target as Y__

In [7]:
X = data.drop(columns=['PRICE', 'PRICE_TRANSFORMED'])
y = data['PRICE_TRANSFORMED']

# Drop features not relevant for modelling

In [8]:
#Drop ASSETID as it is only an identifier, drop ZIP_CODE as the base model only includes base features
X = X.drop(columns=['ASSETID', 'NEIGHBORHOOD', 'DISTANCE_CLUSTER'])

# Splitting the data into training and test set

In [9]:
# Split the dataset into training and testing set with 7:3 ratio
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Check the shapes of the resulting datasets
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (105074, 47)
X_test shape: (45033, 47)
y_train shape: (105074,)
y_test shape: (45033,)


# Target encoding of ZIP CODE

In [10]:
zipcode_means = y_train.groupby(X_train['ZIP_CODE']).mean()

X_train['ZIP_CODE_ENCODED'] = X_train['ZIP_CODE'].map(zipcode_means)
X_test['ZIP_CODE_ENCODED'] = X_test['ZIP_CODE'].map(zipcode_means)

# Drop the ZIP_CODE column from both training and testing sets
X_train = X_train.drop(columns=['ZIP_CODE'])
X_test = X_test.drop(columns=['ZIP_CODE'])

# Inspect data before running the model

In [11]:
X_test.head()

,CONSTRUCTEDAREA,ROOMNUMBER,BATHNUMBER,AMENITYID,HASPARKINGSPACE,PARKINGSPACEPRICE,HASTERRACE,HASLIFT,HASAIRCONDITIONING,HASNORTHORIENTATION,...,PERIOD_201809,PERIOD_201812,MOTORWAY/PRIMARY,OTHER,PEDESTRIAN,SECONDARY/TERTIARY,Madrid,Valencia,Barcelona,ZIP_CODE_ENCODED
32056,87,3,1,3,1,1.0,0,1,1,0,...,1,0,False,True,False,False,True,False,False,12.899613
147130,110,3,2,3,0,1.0,1,1,1,0,...,0,1,False,False,True,False,False,False,True,12.690921
35420,137,4,2,3,1,1.0,0,1,1,1,...,1,0,False,True,False,False,True,False,False,12.928350
128474,81,3,2,3,0,1.0,1,1,1,1,...,0,1,False,True,False,False,False,False,True,12.757758
149036,195,3,2,3,0,1.0,1,1,0,0,...,0,1,False,False,True,False,False,False,True,12.690921


# Test the model with new feature

__Define the best parameters found by GridSearchCV in Notebook 5.1__

In [12]:
best_params = {
    'colsample_bytree': 0.8,
    'learning_rate': 0.1,
    'max_depth': 10,
    'n_estimators': 500,
    'subsample': 0.9
}

__Running the optimized model new feature__

In [13]:
# Initialize XGBoost model with the best parameters
xgb_model = xgb.XGBRegressor(**best_params)

# Train the model on the Box-Cox transformed target
xgb_model.fit(X_train, y_train)

# Predictions in Box-Cox transformed scale
y_pred_boxcox_xgb = xgb_model.predict(X_test)

# Transform predictions back to the original scale
y_pred_xgb = inv_boxcox(y_pred_boxcox_xgb, specified_lambda)
y_test_price = inv_boxcox(y_test, specified_lambda)

# Evaluation metrics on the original scale
mae_xgb = mean_absolute_error(y_test_price, y_pred_xgb)
mse_xgb = mean_squared_error(y_test_price, y_pred_xgb)
rmse_xgb = np.sqrt(mse_xgb)
mape_xgb = mean_absolute_percentage_error(y_test_price, y_pred_xgb)

# Evaluation metrics on the Box-Cox transformed scale
mae_boxcox_xgb = mean_absolute_error(y_test, y_pred_boxcox_xgb)
mse_boxcox_xgb = mean_squared_error(y_test, y_pred_boxcox_xgb)
rmse_boxcox_xgb = np.sqrt(mse_boxcox_xgb)
mape_boxcox_xgb = mean_absolute_percentage_error(y_test, y_pred_boxcox_xgb)

# Print the metrics
print("Metrics on the original PRICE scale:")
print(f"MAE: {mae_xgb:.4f}")
print(f"MSE: {mse_xgb:.4f}")
print(f"RMSE: {rmse_xgb:.4f}")
print(f"MAPE: {mape_xgb:.4f}")

print("\nMetrics on the Box-Cox transformed scale:")
print(f"MAE: {mae_boxcox_xgb:.4f}")
print(f"MSE: {mse_boxcox_xgb:.4f}")
print(f"RMSE: {rmse_boxcox_xgb:.4f}")
print(f"MAPE: {mape_boxcox_xgb:.4f}")

Metrics on the original PRICE scale:
MAE: 43049.8489
MSE: 8058021950.0155
RMSE: 89766.4857
MAPE: 0.1264

Metrics on the Box-Cox transformed scale:
MAE: 0.1229
MSE: 0.0306
RMSE: 0.1751
MAPE: 0.0099


# Differentiate MAPE metric by city and distance to city center

- If we run the combined model: Differentiate MAPE by both city and cluster
- If we run the model for a specific city only: Differentiate MAPE only by cluster

In [14]:
# Create a dataframe with test data indices and predictions
results = pd.DataFrame({
    'y_test_price': y_test_price,
    'y_pred_xgb': y_pred_xgb
}, index=X_test.index)

# Merge the results with the original test set to get city and distance cluster information
if city.lower() == 'combined':
    results = results.merge(data[['Madrid', 'Valencia', 'Barcelona', 'DISTANCE_CLUSTER']], left_index=True, right_index=True)
else:
    results = results.merge(data[['DISTANCE_CLUSTER']], left_index=True, right_index=True)

# Function to calculate MAPE
def calculate_mape(group):
    return mean_absolute_percentage_error(group['y_test_price'], group['y_pred_xgb'])

# Initialize a dictionary to hold MAPE results
mape_results = {}

# Check if the city is 'Combined'
if city.lower() == 'combined':
    # Loop over each city and distance cluster to calculate MAPE
    for city_name in ['Madrid', 'Valencia', 'Barcelona']:
        city_data = results[results[city_name] == True]
        for cluster in city_data['DISTANCE_CLUSTER'].unique():
            cluster_data = city_data[city_data['DISTANCE_CLUSTER'] == cluster]
            mape_results[f"{city_name}_{cluster}"] = calculate_mape(cluster_data)
else:
    # Calculate MAPE for each distance cluster without differentiating by city
    for cluster in results['DISTANCE_CLUSTER'].unique():
        cluster_data = results[results['DISTANCE_CLUSTER'] == cluster]
        mape_results[f"{cluster}"] = calculate_mape(cluster_data)

# Print the MAPE results
for key, value in mape_results.items():
    print(f"MAPE for {key}: {value:.4f}")

MAPE for Madrid_MEDIUM_DISTANCE_TO_CENTER: 0.1263
MAPE for Madrid_LOW_DISTANCE_TO_CENTER: 0.1312
MAPE for Madrid_HIGH_DISTANCE_TO_CENTER: 0.1186
MAPE for Valencia_MEDIUM_DISTANCE_TO_CENTER: 0.1360
MAPE for Valencia_HIGH_DISTANCE_TO_CENTER: 0.1423
MAPE for Valencia_LOW_DISTANCE_TO_CENTER: 0.1497
MAPE for Barcelona_MEDIUM_DISTANCE_TO_CENTER: 0.1142
MAPE for Barcelona_LOW_DISTANCE_TO_CENTER: 0.1225
MAPE for Barcelona_HIGH_DISTANCE_TO_CENTER: 0.1192


If the combined model is run, also print the MAPE for each distance cluster

In [15]:
# Create a dataframe with test data indices and predictions
results = pd.DataFrame({
    'y_test_price': y_test_price,
    'y_pred_xgb': y_pred_xgb
}, index=X_test.index)

# Merge the results with the original test set to get distance cluster information
results = results.merge(data[['DISTANCE_CLUSTER']], left_index=True, right_index=True)

# Function to calculate MAPE
def calculate_mape(group):
    return mean_absolute_percentage_error(group['y_test_price'], group['y_pred_xgb'])

# Initialize a dictionary to hold MAPE results for each distance cluster
cluster_mape_results = {}

# Calculate MAPE for each distance cluster
for cluster in results['DISTANCE_CLUSTER'].unique():
    cluster_data = results[results['DISTANCE_CLUSTER'] == cluster]
    cluster_mape_results[cluster] = calculate_mape(cluster_data)

# Print the MAPE results for each distance cluster
for cluster, mape in cluster_mape_results.items():
    print(f"Distance Cluster: {cluster}")
    print(f"  MAPE: {mape:.4f}")

Distance Cluster: MEDIUM_DISTANCE_TO_CENTER
  MAPE: 0.1243
Distance Cluster: LOW_DISTANCE_TO_CENTER
  MAPE: 0.1319
Distance Cluster: HIGH_DISTANCE_TO_CENTER
  MAPE: 0.1231


If the combined model is run, also print each of the metrics per city

In [16]:
# Create a dataframe with test data indices and predictions
results = pd.DataFrame({
    'y_test_price': y_test_price,
    'y_pred_xgb': y_pred_xgb
}, index=X_test.index)

# Merge the results with the original test set to get city and distance cluster information
if city.lower() == 'combined':
    results = results.merge(data[['Madrid', 'Valencia', 'Barcelona', 'DISTANCE_CLUSTER']], left_index=True, right_index=True)
else:
    results = results.merge(data[['DISTANCE_CLUSTER']], left_index=True, right_index=True)

# Function to calculate MAPE
def calculate_mape(group):
    return mean_absolute_percentage_error(group['y_test_price'], group['y_pred_xgb'])

# Initialize a dictionary to hold MAPE results
mape_results = {}

# Check if the city is 'Combined'
if city.lower() == 'combined':
    # Loop over each city to calculate overall MAPE for each city
    for city_name in ['Madrid', 'Valencia', 'Barcelona']:
        city_data = results[results[city_name] == True]
        mape_results[city_name] = calculate_mape(city_data)

# Print the MAPE results
for key, value in mape_results.items():
    print(f"MAPE for {key}: {value:.4f}")

MAPE for Madrid: 0.1254
MAPE for Valencia: 0.1427
MAPE for Barcelona: 0.1186


# Feature importance of final model

In [17]:
# Permutation importance
perm_importance = permutation_importance(xgb_model, X_test, y_test, random_state=42)
xgb_perm_importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': perm_importance.importances_mean
}).sort_values(by='importance', ascending=False)

# Print results
print("XGBoost Permutation Importance:")
print(xgb_perm_importance_df)

XGBoost Permutation Importance:
                    feature  importance
0           CONSTRUCTEDAREA    0.422682
34                 LATITUDE    0.209933
46         ZIP_CODE_ENCODED    0.106376
30  DISTANCE_TO_CITY_CENTER    0.053692
33                LONGITUDE    0.024058
32  DISTANCE_TO_MAIN_STREET    0.021330
2                BATHNUMBER    0.018563
7                   HASLIFT    0.014405
22               FLOORCLEAN    0.013741
24      CADCONSTRUCTIONYEAR    0.009152
26       CADASTRALQUALITYID    0.007334
31        DISTANCE_TO_METRO    0.005965
15          HASSWIMMINGPOOL    0.005456
28            BUILTTYPEID_2    0.005166
1                ROOMNUMBER    0.004742
25         CADDWELLINGCOUNT    0.003594
4           HASPARKINGSPACE    0.003322
8        HASAIRCONDITIONING    0.002995
6                HASTERRACE    0.002663
23      CADMAXBUILDINGFLOOR    0.001983
27            BUILTTYPEID_1    0.001638
20          HASEXTERNALVIEW    0.001080
38            PERIOD_201812    0.001056
35      

# Run regression on final features due to interpretability

__Prepare data for regression__

We run the regression without transforming the target variable PRICE for full interpretability of the coefficients

In [18]:
#Retransform y_train and y_test to PRICE by taking the exponent and assign to new variable for regression only
y_train_price = np.exp(y_train)
y_test_price = np.exp(y_test)
y_train_price.head()

144534    225000.0
5124      268000.0
68616     356000.0
44359     405000.0
93477     461500.0
Name: PRICE_TRANSFORMED, dtype: float64

In [19]:
#Check that re-transformation to PRICE was done correctly by looking at example
# Access the line with index 5124
row_5124 = data.loc[5124]

# Check the PRICE value and print the message
if row_5124['PRICE'] == 268000:
    print("Retransformation done correctly")
else:
    print("Retransformation incorrect")

Retransformation done correctly


In [20]:
#Change boolean columns to integer to enable analysis of coefficients
bool_columns = X_train.select_dtypes(include=['bool']).columns
X_train[bool_columns] = X_train[bool_columns].astype(int)

bool_columns = X_test.select_dtypes(include=['bool']).columns
X_test[bool_columns] = X_test[bool_columns].astype(int)

In [21]:
X_train.head()

,CONSTRUCTEDAREA,ROOMNUMBER,BATHNUMBER,AMENITYID,HASPARKINGSPACE,PARKINGSPACEPRICE,HASTERRACE,HASLIFT,HASAIRCONDITIONING,HASNORTHORIENTATION,...,PERIOD_201809,PERIOD_201812,MOTORWAY/PRIMARY,OTHER,PEDESTRIAN,SECONDARY/TERTIARY,Madrid,Valencia,Barcelona,ZIP_CODE_ENCODED
144534,78,3,1,3,0,1.0,0,1,0,0,...,1,0,0,0,1,0,0,0,1,12.478790
5124,68,1,1,3,0,1.0,0,1,0,0,...,0,0,0,1,0,0,1,0,0,13.232975
68616,120,3,2,3,1,1.0,1,1,0,0,...,0,1,0,0,0,1,1,0,0,12.191415
44359,106,2,2,3,0,1.0,0,0,1,1,...,0,1,0,1,0,0,1,0,0,13.118411
93477,223,5,3,1,0,1.0,0,1,0,0,...,0,0,1,0,0,0,0,1,0,11.818849


In [22]:
# Drop LONGITUDE, LATITUDE, and ROOM_NUMBER due to high correlation with other features
X_train = X_train.drop(columns=['LONGITUDE', 'LATITUDE', 'ROOMNUMBER'])
X_test = X_test.drop(columns=['LONGITUDE', 'LATITUDE', 'ROOMNUMBER'])

__Run lasso regression for inherent feature selection__

In [23]:
# Feature selection using Lasso
lasso = Lasso(alpha=1000, max_iter=10000) # high alpha due to our high number of features
lasso.fit(X_train, y_train_price)

# Select features with non-zero coefficients
selected_features = X_train.columns[(lasso.coef_ != 0)]

# Filter the datasets to keep only the selected features
X_train_lasso_selected = X_train[selected_features]
X_test_lasso_selected = X_test[selected_features]

# Train the Linear Regression model on the selected features
lr_model = LinearRegression()
lr_model.fit(X_train_lasso_selected, y_train_price)

# Use statsmodels to get coefficients and p-values
X_train_sm = sm.add_constant(X_train_lasso_selected)  # Add constant (intercept) term
lr_model_sm = sm.OLS(y_train_price, X_train_sm).fit()

# Extract coefficients and p-values
coefficients = lr_model_sm.params
p_values = lr_model_sm.pvalues

# Predict the values on the test set
y_pred_lr_price = lr_model.predict(X_test_lasso_selected)

# Evaluation metrics on the original scale
mae_lr_price = mean_absolute_error(y_test_price, y_pred_lr_price)
mse_lr_price = mean_squared_error(y_test_price, y_pred_lr_price)
rmse_lr_price = np.sqrt(mse_lr_price)
mape_lr_price = mean_absolute_percentage_error(y_test_price, y_pred_lr_price)
r2_lr_price = r2_score(y_test_price, y_pred_lr_price)

# Calculate adjusted R-squared
n = X_test_lasso_selected.shape[0]
p = X_test_lasso_selected.shape[1]
adjusted_r2_lr_price = 1 - (1 - r2_lr_price) * (n - 1) / (n - p - 1)

# Print the results
print("Selected Features:", selected_features)

print("\nCoefficients and P-values:")
for coef, p_val in zip(coefficients.index, p_values):
    print(f"{coef}: Coefficient = {coefficients[coef]}, P-value = {p_values[coef]}")

print("\nEvaluation Metrics:")
print("Mean Absolute Error (MAE):", mae_lr_price)
print("Mean Squared Error (MSE):", mse_lr_price)
print("Root Mean Squared Error (RMSE):", rmse_lr_price)
print("Mean Absolute Percentage Error (MAPE):", mape_lr_price)
print("R-squared (R2):", r2_lr_price)
print("Adjusted R-squared:", adjusted_r2_lr_price)

Selected Features: Index(['CONSTRUCTEDAREA', 'BATHNUMBER', 'HASPARKINGSPACE', 'PARKINGSPACEPRICE',
       'HASLIFT', 'HASAIRCONDITIONING', 'HASEASTORIENTATION', 'HASBOXROOM',
       'HASWARDROBE', 'HASSWIMMINGPOOL', 'HASDOORMAN', 'HASEXTERNALVIEW',
       'FLOORCLEAN', 'CADMAXBUILDINGFLOOR', 'CADCONSTRUCTIONYEAR',
       'CADDWELLINGCOUNT', 'CADASTRALQUALITYID', 'BUILTTYPEID_2',
       'DISTANCE_TO_CITY_CENTER', 'DISTANCE_TO_METRO',
       'DISTANCE_TO_MAIN_STREET', 'PERIOD_201803', 'PERIOD_201812',
       'PEDESTRIAN', 'SECONDARY/TERTIARY', 'Madrid', 'Valencia',
       'ZIP_CODE_ENCODED'],
      dtype='object')

Coefficients and P-values:
const: Coefficient = -1496870.0775543014, P-value = 1.04096004986999e-260
CONSTRUCTEDAREA: Coefficient = 3880.0737865948836, P-value = 0.0
BATHNUMBER: Coefficient = 38050.235160925, P-value = 0.0
HASPARKINGSPACE: Coefficient = 16820.09557056737, P-value = 1.0289594701334533e-25
PARKINGSPACEPRICE: Coefficient = -0.4209337778101059, P-value = 0.0003255